In [1]:
import numpy as np 
import pandas as pd 
import os
import shutil

from glob import glob 
from skimage.io import imread
import gc

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Using TensorFlow backend.


In [ ]:
print(os.listdir("C:/Users/arvin/Documents/Lymph Nodes/all/"))

In [2]:
base_tile_dir = 'C:/Users/arvin/Documents/Lymph Nodes/all/train_1/'
df = pd.DataFrame({'path': glob(os.path.join(base_tile_dir,'*.tif'))})
df['id'] = df.path.map(lambda x: x.split('/')[6].split("\\")[1].split(".")[0])
labels = pd.read_csv("C:/Users/arvin/Documents/Lymph Nodes/all/train_labels.csv")
df_data = df.merge(labels, on = "id")

# removing this image because it caused a training error previously
df_data = df_data[df_data['id'] != 'dd6dfed324f9fcb6f93f46f32fc800f2ec196be2']

# removing this image because it's black
df_data = df_data[df_data['id'] != '9369c7278ec8bcc6c880d99194de09fc2bd4efbe']
df_data.head(3)

,path,id,label
0,C:/Users/arvin/Documents/Lymph Nodes/all/train...,00001b2b5609af42ab0ab276dd4cd41c3e7745b5,1
1,C:/Users/arvin/Documents/Lymph Nodes/all/train...,000020de2aa6193f4c160e398a8edea95b1da598,0
2,C:/Users/arvin/Documents/Lymph Nodes/all/train...,00004aab08381d25d315384d646f5ce413ea24b1,0


In [3]:
SAMPLE_SIZE = 80000 # load 80k negative examples

# take a random sample of class 0 with size equal to num samples in class 1
df_0 = df_data[df_data['label'] == 0].sample(SAMPLE_SIZE, random_state = 101)
# filter out class 1
df_1 = df_data[df_data['label'] == 1].sample(SAMPLE_SIZE, random_state = 101)

# concat the dataframes
df_data = shuffle(pd.concat([df_0, df_1], axis=0).reset_index(drop=True))

# train_test_split # stratify=y creates a balanced validation set.
y = df_data['label']
df_train, df_val = train_test_split(df_data, test_size=0.10, random_state=101, stratify=y)

In [4]:
df_data.set_index('id', inplace=True)
df_data.head()

,path,label
id,,
56731dbd353d400ba9618bac7b41db9b25b479f8,C:/Users/arvin/Documents/Lymph Nodes/all/train...,1
07de2d2499ea93fb580984deefd2fd1139a3b7c3,C:/Users/arvin/Documents/Lymph Nodes/all/train...,0
8bc558435b01b5eb342c7d2e0535f26997976f2a,C:/Users/arvin/Documents/Lymph Nodes/all/train...,0
31b73ff3c451f3f1817aff5326a7d68df35cb8e1,C:/Users/arvin/Documents/Lymph Nodes/all/train...,0
e44fae776983ab89254245824b1f202ed4713185,C:/Users/arvin/Documents/Lymph Nodes/all/train...,0


In [5]:
train_path = 'base_dir/train_split'
valid_path = 'base_dir/valid_split'

In [11]:
from keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = 96
num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 32
val_batch_size = 32

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

datagen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x,
                            #width_shift_range = 0.1,
                            #height_shift_range = 0.1,
                            #zoom_range = 0.2,
                            horizontal_flip=True,
                            vertical_flip=True)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='binary')

val_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='binary')

# Note: shuffle=False causes the test dataset to not be shuffled
test_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='binary',
                                        shuffle=False)

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [ ]:
from keras import applications
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, GlobalMaxPool2D, Concatenate
from keras.optimizers import RMSprop, Adam

model = applications.nasnet.NASNetMobile(weights = None, include_top=False, input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3))
#for layer in model.layers[:5]:
    #layer.trainable = False

x = model.output
x1 = GlobalMaxPool2D()(x)
x2 = GlobalAveragePooling2D()(x)
x3 = Flatten()(x)
x = Concatenate(axis=-1)([x1,x2,x3])
#x = Flatten()(x)
#x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics=["accuracy"]) 

In [17]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [18]:
checkpoint = ModelCheckpoint("nasnet_1_8.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
history = model_final.fit_generator(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=8,
                   callbacks=[checkpoint, early])

Epoch 1/8
 149/4500 [..............................] - ETA: 46:04 - loss: 8.0187 - acc: 0.5002

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

# make a prediction
y_pred_keras = model_final.predict_generator(test_gen, steps=len(df_val), verbose=1)
#fpr_keras, tpr_keras, thresholds_keras = roc_curve(test_gen.classes, y_pred_keras)
#auc_keras = auc(fpr_keras, tpr_keras)
#auc_keras
y_pred_keras

In [ ]:
fpr_keras, tpr_keras, thresholds_keras = roc_curve(test_gen.classes, y_pred_keras)
auc_keras = auc(fpr_keras, tpr_keras)
auc_keras

In [ ]:
y_eval = model_final.evaluate_generator(test_gen)

In [ ]:
y_eval

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='area = {:.3f}'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()